In [1]:
import numpy
from sklearn.metrics import accuracy_score

from tslearn.generators import random_walk_blobs
from tslearn.preprocessing import TimeSeriesScalerMinMax, \
    TimeSeriesScalerMeanVariance
from tslearn.neighbors import KNeighborsTimeSeriesClassifier, \
    KNeighborsTimeSeries
import pandas as pd
import numpy as np
import random
from tslearn.datasets import UCR_UEA_datasets
from tslearn.preprocessing import TimeSeriesResampler


In [2]:
df = pd.read_pickle("Train.pkl")

In [12]:
randomlist1 = random.sample(range(0, 260), 78)

df_test1 = df.loc[df["Batch"].isin(randomlist1)]
df_train1 = df.loc[~df["Batch"].isin(randomlist1)]

In [13]:
df_steps1 = df_test1.loc[df_test1["CuStepNo ValueY"] != df_test1["Next_Step"]]
df_steps2 = df_train1.loc[df_train1["CuStepNo ValueY"] != df_train1["Next_Step"]]

In [14]:
y_train1 = df_steps2["DeviationID ValueY"].astype(int)
y_test1 = df_steps1["DeviationID ValueY"].astype(int)
X_train1 = df_train1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)
X_test1 = df_test1.drop(["DeviationID ValueY","Next_Step","timestamp","Unnamed: 0"],axis=1)

In [148]:
y_train1

0         1
248       1
321       1
854       1
2102      5
         ..
264688    9
264886    9
265190    9
265261    9
265798    9
Name: DeviationID ValueY, Length: 700, dtype: int32

In [30]:
X_train2 = []
for i in X_train1.Batch.unique():
    for j in X_train1["CuStepNo ValueY"].unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
        alle = np.linspace(0, num.shape[0]-1, 40).round()
        alle = [int(i) for i in alle]
        num = [num[i] for i in alle]
        # num = np.atleast_3d(num)
        # num = num.reshape(1, 40, 40)
        X_train2.append(num)

In [122]:
X_train2.shape

(700, 40, 40)

In [89]:
np.linspace(0, 5, 40).round()

array([0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2.,
       2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4.,
       4., 4., 5., 5., 5., 5.])

In [88]:
b = 200
for i in X_train1.Batch.unique():
    for j in X_train1["CuStepNo ValueY"].unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
        a = num.shape[0]
        if a < b:
            b = a
            print(num.shape)

(73, 40)
(66, 40)
(65, 40)
(53, 40)
(5, 40)


In [38]:
X_train2 = np.array(X_train2)

In [149]:
X_train = []
for i in X_train1.Batch.unique():
    for j in X_train1["CuStepNo ValueY"].unique():
        num = numpy.array(X_train1.loc[(X_train1["Batch"] == i)&(X_train1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
        # num = np.atleast_3d(num)
        # num = num.reshape(1, num.shape[0], 40)
        X_train.append(num)

In [156]:
X_test = []
for i in X_test1.Batch.unique():
    for j in X_test1["CuStepNo ValueY"].unique():
        num = numpy.array(X_test1.loc[(X_test1["Batch"] == i)&(X_test1["CuStepNo ValueY"] == j)].drop(["Batch"],axis=1))
        # num = np.atleast_3d(num)
        # num = num.reshape(1, num.shape[0], 40)
        X_test.append(num)

In [158]:
X_train4 = TimeSeriesResampler(sz=100).fit_transform(X_train)

In [164]:
X_test4 = TimeSeriesResampler(sz=100).fit_transform(X_test)

In [168]:
scaler = TimeSeriesScalerMinMax(value_range=(0., 1.))  # Rescale time series
X_scaled = scaler.fit_transform(X_train4)
X_test_scaled = scaler.transform(X_test4)

In [163]:
X_test4.shape

(700, 100, 40)

In [170]:
knn = KNeighborsTimeSeriesClassifier(n_neighbors=3, metric="dtw")
knn.fit(X_train4, y_train1) 
y_pred = knn.predict(X_test4)
accuracy_score(y_test1, y_pred)

0.7368421052631579

In [126]:
from tslearn.metrics import dtw_path

In [140]:
path, dist = dtw_path(X_train[0], X_train[4])

In [143]:
path.__len__()

327

In [141]:
dist

498.3356227988018

In [144]:
X_train[0].__len__()

248

In [132]:
path.__len__()

548

In [124]:
X_train[0]

array([[1.00000000e+00, 1.49985779e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.71031873e-03],
       [1.00000000e+00, 1.49985779e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.71468942e-03],
       [1.00000000e+00, 1.50027740e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.71223748e-03],
       ...,
       [1.00000000e+00, 1.36731201e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74658047e-03],
       [1.00000000e+00, 1.36644073e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74658047e-03],
       [1.00000000e+00, 1.36576385e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74411153e-03]])

In [55]:
from sklearn.utils.estimator_checks import _NotAnArray as NotAnArray
# from sklearn.utils.estimator_checks import NotAnArray

In [59]:
np.array(X_train[0]).ndim == 0

False

In [90]:
from tslearn.utils import to_time_series
from tslearn.utils import to_time_series_dataset
from tslearn.utils import check_equal_size
from tslearn.utils import ts_size
from scipy import interpolate

In [65]:
to_time_series(X_train[1], remove_nans=True)

array([[7.00000000e+00, 1.36526093e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74385821e-03],
       [7.00000000e+00, 1.36503082e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.72162382e-03],
       [7.00000000e+00, 1.36571106e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.70616596e-03],
       ...,
       [7.00000000e+00, 1.41533936e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.75515050e-03],
       [7.00000000e+00, 1.41488831e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74055621e-03],
       [7.00000000e+00, 1.41446075e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74304610e-03]])

In [72]:
X_ = to_time_series_dataset(X_train)

In [74]:
X_.shape

(700, 615, 40)

In [110]:
ts_size(X_[5])

66

In [111]:
X_[5, :66, 4]

array([1.70543981, 1.74942136, 1.68229175, 1.65451384, 1.65451384,
       1.64467597, 1.71354175, 1.71238422, 1.71006942, 1.63020825,
       1.63483787, 1.63773155, 1.70717597, 1.70717597, 1.70196772,
       1.61921287, 1.62210655, 1.69849539, 1.69965291, 1.69733787,
       1.69039345, 1.62615752, 1.62962961, 1.625     , 1.70196772,
       1.71122694, 1.71064806, 1.64699078, 1.63252306, 1.62731481,
       1.64641213, 1.70775461, 1.70023155, 1.70081019, 1.63773155,
       1.63020825, 1.62847233, 1.67708325, 1.71643519, 1.71354175,
       1.70254636, 1.63368058, 1.62962961, 1.68287039, 1.70775461,
       1.70543981, 1.70601845, 1.63657403, 1.63657403, 1.640625  ,
       1.70196772, 1.71006942, 1.71180558, 1.70775461, 1.640625  ,
       1.63136578, 1.63657403, 1.70659733, 1.70833325, 1.703125  ,
       1.70543981, 1.64120364, 1.63020825, 1.62673616, 1.69675922,
       1.70891213])

In [112]:
f = interpolate.interp1d(numpy.linspace(0, 1, 66), X_[5, :66, 4],
                             kind="slinear")

In [119]:
xnew = numpy.linspace(0, 1, 34)

In [120]:
f(xnew)

array([1.70543981, 1.68432598, 1.65451384, 1.70728122, 1.71035   ,
       1.63413641, 1.69454971, 1.7030725 , 1.62140505, 1.69933722,
       1.69249782, 1.62847225, 1.67397946, 1.7108761 , 1.63866088,
       1.63773153, 1.7038791 , 1.67022661, 1.6294192 , 1.69377802,
       1.70921023, 1.6322075 , 1.69116513, 1.70561515, 1.63657403,
       1.65549596, 1.7104377 , 1.69554923, 1.63215491, 1.70680774,
       1.70333544, 1.64053725, 1.62885807, 1.70891213])

In [76]:
equal_size = check_equal_size(X_)

In [78]:
numpy.linspace(0, 1, 40)

array([0.        , 0.02564103, 0.05128205, 0.07692308, 0.1025641 ,
       0.12820513, 0.15384615, 0.17948718, 0.20512821, 0.23076923,
       0.25641026, 0.28205128, 0.30769231, 0.33333333, 0.35897436,
       0.38461538, 0.41025641, 0.43589744, 0.46153846, 0.48717949,
       0.51282051, 0.53846154, 0.56410256, 0.58974359, 0.61538462,
       0.64102564, 0.66666667, 0.69230769, 0.71794872, 0.74358974,
       0.76923077, 0.79487179, 0.82051282, 0.84615385, 0.87179487,
       0.8974359 , 0.92307692, 0.94871795, 0.97435897, 1.        ])

In [77]:
equal_size

False

In [63]:
X_train[1]

array([[7.00000000e+00, 1.36526093e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74385821e-03],
       [7.00000000e+00, 1.36503082e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.72162382e-03],
       [7.00000000e+00, 1.36571106e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.70616596e-03],
       ...,
       [7.00000000e+00, 1.41533936e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.75515050e-03],
       [7.00000000e+00, 1.41488831e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74055621e-03],
       [7.00000000e+00, 1.41446075e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74304610e-03]])

In [60]:
dataset_out = numpy.zeros((700, 300, 40), dtype=float) + numpy.nan

In [61]:
dataset_out

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

In [51]:
X_train

[array([[1.00000000e+00, 1.49985779e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.71031873e-03],
        [1.00000000e+00, 1.49985779e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.71468942e-03],
        [1.00000000e+00, 1.50027740e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.71223748e-03],
        ...,
        [1.00000000e+00, 1.36731201e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.74658047e-03],
        [1.00000000e+00, 1.36644073e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.74658047e-03],
        [1.00000000e+00, 1.36576385e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.74411153e-03]]),
 array([[7.00000000e+00, 1.36526093e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.74385821e-03],
        [7.00000000e+00, 1.36503082e+02, 1.00000000e+02, ...,
         2.00000003e-01, 0.00000000e+00, 1.72162382e-03],
        [7.00000000e+00, 

In [22]:
X_train = TimeSeriesResampler(sz=40).fit_transform(X_train)

In [25]:
np.array(X_train).shape

(700, 40, 40)

In [46]:
X_train2[0]

array([[1.00000000e+00, 1.49985779e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.71031873e-03],
       [1.00000000e+00, 1.49905792e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.70696317e-03],
       [1.00000000e+00, 1.49691193e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.66326272e-03],
       ...,
       [1.00000000e+00, 1.37491425e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.76144601e-03],
       [1.00000000e+00, 1.37267242e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.71736395e-03],
       [1.00000000e+00, 1.36576385e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74411153e-03]])

In [47]:
X_train[0]

array([[1.00000000e+00, 1.49985779e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.71031873e-03],
       [1.00000000e+00, 1.49883372e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.69186899e-03],
       [1.00000000e+00, 1.49701111e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.66326272e-03],
       ...,
       [1.00000000e+00, 1.37475342e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.76144601e-03],
       [1.00000000e+00, 1.37282135e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.72276717e-03],
       [1.00000000e+00, 1.36576385e+02, 1.00000000e+02, ...,
        2.00000003e-01, 0.00000000e+00, 1.74411153e-03]])